## The need and implementation of QTensor class

With only tequila objectives defined, it was a time-consuming process for the user to implement a vector, matrix or a tensor of expectation values (or tequila objectives) and manipulate it.
Thus there was a need of a new class that gives the ease of creating these objects and doing operations with them efficiently. 
We call this new class as `QTensor`.<br>
Below we first demonstrate how, without the QTensor, one could create the above mentioned mathematical objects and do operations with them. Thereafter, we show using QTensor class the ease of doing the same.

In [1]:
import tequila as tq
import numpy as np
from numpy import pi

[Qibo 0.1.6|INFO|2022-01-14 10:46:45]: Using numpy backend on /CPU:0
[Qibo 0.1.6|WARNING|2022-01-14 10:46:45]: numpy backend uses `np.einsum` and supports CPU only. Consider installing the qibojit or qibotf backends for increased performance and to enable GPU acceleration.


Without the QTensor class, the usual strategy would be to create a class with tequila objectives as primitives and initialize it with a desired shape. To this class then, we will add methods to access the elements given the index number.
Let's say we we call this new class `tequialTensor`. We can define it as:

In [2]:
class tequilaTensor:
    def __init__(self,objective_list,shape):
        self.shape = shape
        self.objective_list = objective_list

    def access(self, idx): 
        if len(idx) != len(self.shape):
            raise Exception("access: shapes don't match {} vs {}".format(idx, self.shape))
        else:
            for j in range(len(idx)):
                if idx[j] >= self.shape[j]:
                    raise ValueError("no such matrix cell exists")
            list_index = 0
            for i in range(len(idx)):
                if i != len(idx) -1 :
                    list_index += idx[i]*(self._prod(self.shape,i))
                else:
                    list_index += (idx[i] + 1)
            return self.objective_list[list_index-1]

    def _prod(self, any_list, integer):
        prod = 1
        for i in range(integer+1, len(any_list)):
            prod = prod * any_list[i]
        return prod
    

Using the above `access` method, one can access any element of the given tensor. 
Let us briefly see how it works. First, this method checks whether we the index that we have provided exists in that tensor or not. Once it's confirmed if there is the index number provided is correct, we then run a small loop to output the element at the desired index number.

To make accessing elements easier for the user, we add another method `__getitem__` which essentially calls `access`. This gives us the priviledge of using `[]` to access the elements.<br>
Next, we want `tequilaTensor` class to be able to add two tensors of the same shape and return an error if the shapes are not same. Thus, we do the following:

In [3]:
class tequilaTensor:
    def __init__(self,objective_list,shape):
        self.shape = shape
        self.objective_list = objective_list

    def access(self, idx): 
        if len(idx) != len(self.shape):
            raise Exception("access: shapes don't match {} vs {}".format(idx, self.shape))
        else:
            for j in range(len(idx)):
                if idx[j] >= self.shape[j]:
                    raise ValueError("no such matrix cell exists")
            list_index = 0
            for i in range(len(idx)):
                if i != len(idx) -1 :
                    list_index += idx[i]*(self._prod(self.shape,i))
                else:
                    list_index += (idx[i] + 1)
            return self.objective_list[list_index-1]
    
    def __getitem__(self,item):
        return self.access(item)

    def __add__(self, other):
        if self.shape != other.shape:
            raise Exception("shapes don't match")
        added_list = []
        for i in range(len(self.objective_list)):
            added_list.append(self.objective_list[i] + other.objective_list[i])
        return tequilaTensor(objective_list=added_list, shape=self.shape)

    def _prod(self, any_list, integer):
        prod = 1
        for i in range(integer+1, len(any_list)):
            prod = prod * any_list[i]
        return prod

In the above code, we first compare the shapes of the two tensors to be added. If the shapes are same, we do element-wise addition, and return the resultant tequilaTensor.

The next desired operation is scalar multiplication and element-wise multiplication. 
We want that our class should be able to perform both left and right multiplications. To do this, we add a `__mul__` method in the following way:

In [4]:
class tequilaTensor:
    def __init__(self,objective_list,shape):
        self.shape = shape
        self.objective_list = objective_list

    def access(self, idx): 
        if len(idx) != len(self.shape):
            raise Exception("access: shapes don't match {} vs {}".format(idx, self.shape))
        else:
            for j in range(len(idx)):
                if idx[j] >= self.shape[j]:
                    raise ValueError("no such matrix cell exists")
            list_index = 0
            for i in range(len(idx)):
                if i != len(idx) -1 :
                    list_index += idx[i]*(self._prod(self.shape,i))
                else:
                    list_index += (idx[i] + 1)
            return self.objective_list[list_index-1]
    
    def __getitem__(self,item):
        return self.access(item)

    def __add__(self, other):
        if self.shape != other.shape:
            raise Exception("shapes don't match")
        added_list = []
        for i in range(len(self.objective_list)):
            added_list.append(self.objective_list[i] + other.objective_list[i])
        return tequilaTensor(objective_list=added_list, shape=self.shape)

    def __mul__(self,other):
        if isinstance(other,float) or isinstance(other, int) or isinstance(other, complex):
            multiplied_list = []
            for i in range(len(self.objective_list)):
                multiplied_list.append(self.objective_list[i] * other)
            return tequilaTensor(objective_list=multiplied_list, shape=self.shape)            
        if self.shape != other.shape:
            raise Exception("shapes don't match")
        multiplied_list = []
        for i in range(len(self.objective_list)):
            multiplied_list.append(self.objective_list[i] * other.objective_list[i])
        return tequilaTensor(objective_list=multiplied_list, shape=self.shape)

    def __rmul__(self,other):
        return self.__mul__(other)

    def _prod(self, any_list, integer):
        prod = 1
        for i in range(integer+1, len(any_list)):
            prod = prod * any_list[i]
        return prod

In the above `__mul__` method, we first check whether the `other` - which is being multiplied to our original `tequilaTensor` object - is an `int`, `float`,`complex`, or a `tequilaTensor` itself. If the `other` is a `tequilaTensor`, we check whether it has the same shape, and if it does, we then do an element-wise multiplication and return the resultant tequilaTensor. 

Having defined these basic operations, let us first test the above class using numbers and then we will test it with tequila objectives.

In [5]:
# Let us first initialize a tequilaTensor with numbers
A = tequilaTensor(objective_list=[1,2,3,4,5,6,7,8],shape=[4,2]) 

# Now let's access and print an element of this tequilaTensor
print('A[3,1]: ', A[3,1])

# Right multiplication
A = A*2
print('after right multiplication with 2, A[3,1]: ', A[3,1])

# Left multiplication
A = 2*A
print('after a left multiplication with 2, A[3,1]: ', A[3,1])

# Product of two objects of tequilaTensor class.
A = A*A
print('Let A = A*A. Then A[3,1]: ', A[3,1])

## If we try to add two objects with wrong, it gives an error as desired.
## To check, uncomment the two lines below and run.
# B = tequilaTensor(objective_list=[3,4],shape=[2])

B = tequilaTensor(objective_list=[3,4,2,3.4, 5,6.7, 8,9.0],shape=[4,2])

C = A+B

print('objective list of A: ',A.objective_list)
print('objective list of B: ',B.objective_list)
print('objective list of C: ',C.objective_list)


C = B*2 
print(B.shape)
print(C.shape)
print(B[0,0])
for i in range(A.shape[0]):
    print(end='\n')
    for j in range(A.shape[1]):
        print(A[i,j],end=' ')

A[3,1]:  8
after right multiplication with 2, A[3,1]:  16
after a left multiplication with 2, A[3,1]:  32
Let A = A*A. Then A[3,1]:  1024
objective list of A:  [16, 64, 144, 256, 400, 576, 784, 1024]
objective list of B:  [3, 4, 2, 3.4, 5, 6.7, 8, 9.0]
objective list of C:  [19, 68, 146, 259.4, 405, 582.7, 792, 1033.0]
[4, 2]
[4, 2]
3

16 64 
144 256 
400 576 
784 1024 

So we verify that our class works well with numbers, and we can initialize a tequilaTensor, access its elements, add two tequilaTensors of same shape, and do scalar multiplication.

After testing the `tequilatensor` class with numbers, now let us test it with `objectives`.
Below we define some basic expectation values which we will use to define an object of `tequilaTensor` class and use the methods we defined in this class.

In [6]:
# the circuit
U0 = tq.gates.Ry(angle = pi/2,target = 0)
H0 = 2*tq.paulis.X(0)
E0 = tq.ExpectationValue(H=H0, U = U0)

print(H0)
print(E0)

+2.0000X(0)
Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled


In [7]:
U = tq.gates.H(0) + tq.gates.X(target=1,control=0)

H1 = tq.paulis.X(0)
H2 = tq.paulis.X([0,1])
H3 = tq.paulis.Z([0,1])
H4 = tq.paulis.Z(1)

E1 = tq.ExpectationValue(H = H1, U=U)
E2 = tq.ExpectationValue(H = H2, U=U)
E3 = tq.ExpectationValue(H = H3, U=U)
E4 = tq.ExpectationValue(H = H4, U=U)
# print(H1)
# print(H2)
# print(H3)
# print(H4)
# print(E2)

Now let us construct some `tequilaTensor` objects of various shapes and use the methods on them

In [8]:
X = tequilaTensor(objective_list=[E1,E2],shape=[2,1])
Y = tequilaTensor(objective_list=[E1,E2,E3,E4],shape=[2,2])

In [9]:
print(tq.simulate(X.objective_list[0]))
print(tq.simulate(Y.objective_list[1]))
print(tq.simulate(Y.objective_list[2]))
print(tq.simulate(Y.objective_list[3]))

0.0
0.9999999999999998
0.9999999999999998
0.0


In [10]:
print(Y[0,0])
print(tq.simulate(Y[0,0]))
print(type(X[0,0]))
print(type(Y))

Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled
0.0
<class 'tequila.objective.objective.Objective'>
<class '__main__.tequilaTensor'>


In [11]:
# Addition
Z = Y+Y 

# Scalar multiplication and element-wise multiplication of two tequilaTensors
Z1 = 2*Y
Z2 = Y*2
Z3 = Y*Y
for i in range(Z3.shape[0]):
    print(end='\n')
    for j in range(Z3.shape[1]):
        print(Z3[i,j],end=' ')


Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled 
Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled Objective with 1 unique expectation values
total measurements = 1
variables          = []
types              = not compiled 

So, in the above examples we see that we are able to do basic operations by forming our own tensor class. 
Now, we want to add more functions and it would be good to have all the operations that can be performed on an `numpy.ndarray` object. Apart from that, we would like to also have `compile` and `simulate`.<br>
Now if we keep adding methods to perform compilation, simulation, more linear algebraic functions, or even applying a function to all elements of the defined tensor, it will become harder or inefficient, and time-consuming for the user.

With this idea, we introduce the `QTensor` class using which the user can easily define and initialize a tensor with tequila objectives as its elements. Moreover, we derive this class from numpy.ndarray class. This means that we are inheriting the properties of numpy.ndarray and don't need to redefine all those linear algbraic functions again.
Besides, using this QTensor class, the user is able to apply any function to the whole QTensor object in the same way as one can apply the function to a tequila objective using the `apply` method. Lastly, we can apply, `grad`, `compile`, and `simulate` in the same way as we do for tequila objectives. <br>
Below we justify this by constructing simple QTensor objects (a (2,2) matrix and a (2,1) vector) and performing some operations on it. More details and usage of QTensor have been provided in the `QTensorTutorial`.

In [12]:
V = tq.QTensor(shape=[2,2])
W = tq.QTensor(shape=[2])
V[0,0] = E1
V[0,1] = E2
V[1,0] = E3
V[1,1] = E4
W[0] = E1
W[1] = E2

## An alternate way to define and initialize QTensor is
## V = tq.QTensor(objective_list=[E1,E2,E3,E4], shape = [2,2])

Using the `print` command, we can get the details of the desired QTensor.

In [13]:
print(V)
print(W)

QTensor of shape (2, 2) with 4 unique expectation values
total measurements = 4
variables          = []
types              = not compiled
QTensor of shape (2,) with 2 unique expectation values
total measurements = 2
variables          = []
types              = not compiled


For linear algebraic operations, we can treat the QTensor object as an ndarray and use all the numpy linear algebra functions. An example is given below

In [14]:
U = np.dot(V,W)
print(type(U))
print(U)

<class 'tequila.objective.qtensor.QTensor'>
QTensor of shape (2,) with 4 unique expectation values
total measurements = 4
variables          = []
types              = not compiled


We can comile and simulate the QTensor objects in the same way we compile tequila objectives.

In [15]:
print('V = \n',tq.simulate(V))
print('W = \n',tq.simulate(W))
print('U = V*W = \n',tq.simulate(U))

V = 
 [[0. 1.]
 [1. 0.]]
W = 
 [0. 1.]
U = V*W = 
 [1. 0.]


Lastly, for applying a function on a QTensor object, the same convention that is used for applying a function on a tequila objective, can be used. For instance, if we want to apply exponential over all elements of the QTensor `V`, then we simply do the following

In [16]:
V1 = V.apply(np.exp)
print(tq.simulate(V1))

[[1.         2.71828183]
 [2.71828183 1.        ]]


Thus, `QTensor` class provides an easy and efficient way to create vectors, matrices and tensors of tequila objectives, and perform operations on it. We can perform all the operations that one can do on a numpy.ndarray object. 